In [1]:
#check if which virtual env i am in, and if env variable for google cloud is saved
import sys
import os
print(f"We are in the virtual env named: {sys.prefix}")
print(f"GOOGLE_APPLICATION_CREDENTIALS = {os.environ['GOOGLE_APPLICATION_CREDENTIALS']}")

We are in the virtual env named: C:\Users\fangn\anaconda3\envs\hackernews
GOOGLE_APPLICATION_CREDENTIALS = C:\Users\fangn\OneDrive\Documents\Data Science Portfolio\hackernews-agtech-2d07274b705e.json


In [34]:
print("Path at terminal when executing this file")
print(os.getcwd() + "\n")

Path at terminal when executing this file
C:\Users\fangn\Documents\hackernews-agtech



In [2]:
import pandas as pd
from google.cloud import bigquery

In [21]:
#this is the method used in the Kaggle notebooks
client = bigquery.Client()

# Using WHERE reduces the amount of data scanned / quota used
query = """
SELECT title, text, type, timestamp
FROM `bigquery-public-data.hacker_news.full`
WHERE REGEXP_CONTAINS(title, r"(a|A)gtech")
ORDER BY time
"""

query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
headlines = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 100 headlines
headlines.head(100)

,title,text,type,timestamp
0,42 New Food Tech and Agtech Funding Opportunit...,None,story,2015-06-08 20:39:10+00:00
1,Agtech Funding Dips to $3.2bn While Deal Activ...,None,story,2017-02-02 22:46:07+00:00
2,Agricultural giants risk being left behind in ...,None,story,2017-05-23 18:59:50+00:00
3,This is the highest funded Agtech Startup: Plenty,None,story,2017-11-10 07:49:03+00:00
4,Beginner's Guide to Agtech: Innovating in the ...,None,story,2018-10-01 07:52:03+00:00
5,Agtech: Innovating in the Farming Industry,None,story,2018-10-05 11:12:13+00:00
6,10 European agtech startups preparing us for t...,None,story,2018-11-13 11:23:27+00:00
7,Agtech startup Farmee to support farmers remotely,None,story,2019-01-14 21:23:16+00:00
8,What Is the Future of Food? Interview with Dav...,None,story,2019-04-02 01:09:11+00:00
9,Overview: Techstars Farm to Fork Demo Day Agtech,None,story,2019-10-16 19:14:21+00:00


In [3]:
#this is another method using pandas to save query results to a df
client = bigquery.Client()

# Using WHERE reduces the amount of data scanned / quota used
query = """
SELECT title, type, timestamp, url
FROM `bigquery-public-data.hacker_news.full`
WHERE REGEXP_CONTAINS(title, r"(a|A)gtech") OR REGEXP_CONTAINS(title, r"(a|A)gritech")
ORDER BY timestamp
"""

agtech_titles_df = client.query(query).to_dataframe()
agtech_titles_df

,title,type,timestamp,url
0,42 New Food Tech and Agtech Funding Opportunit...,story,2015-06-08 20:39:10+00:00,http://www.foodtechconnect.com/2015/06/04/42-n...
1,Amazone radar sensor system wins Agritechnica ...,story,2015-11-29 06:04:26+00:00,http://www.weeklytimesnow.com.au/machine/amazo...
2,Agtech Funding Dips to $3.2bn While Deal Activ...,story,2017-02-02 22:46:07+00:00,https://agfundernews.com/a-year-of-contrasts-a...
3,Agricultural giants risk being left behind in ...,story,2017-05-23 18:59:50+00:00,https://agfundernews.com/agricultural-giants-r...
4,Cool Agritech Solutions for Drones and Autonom...,story,2017-08-03 11:49:29+00:00,https://n-ix.com/5-agritech-solutions-drones-a...
5,What Taizo Son can do for agritech and food st...,story,2017-09-15 04:42:28+00:00,https://www.techinasia.com/taizo-son-starts-ac...
6,This is the highest funded Agtech Startup: Plenty,story,2017-11-10 07:49:03+00:00,https://www.vox.com/energy-and-environment/201...
7,Agritech mobile platform gets grant funding fo...,story,2018-05-16 17:36:31+00:00,https://www.iafrikan.com/2018/05/16/africas-ag...
8,Beginner's Guide to Agtech: Innovating in the ...,story,2018-10-01 07:52:03+00:00,https://softmedialab.com/blog/agtech-innovatin...
9,Agtech: Innovating in the Farming Industry,story,2018-10-05 11:12:13+00:00,https://softmedialab.com/blog/agtech-innovatin...
